In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 30.1 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [ ]:
import numpy as np
import pandas as pd
import os
import gc
from tqdm.auto import tqdm

# ==============================================================================
# 1. CẤU HÌNH ĐƯỜNG DẪN
# ==============================================================================
# Folder chứa Ankh Base (TEMPLATE - Dùng để lấy thứ tự chuẩn)
REF_DIR = '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged'

# Folder chứa Ankh3 Large gốc (SOURCE - Dữ liệu thô)
SRC_DIR = '/content/drive/MyDrive/cafa/features/ankh_large/'

# Folder đầu ra (DESTINATION - Nơi lưu file đã align)
OUTPUT_DIR = '/content/drive/MyDrive/CAFA6_Embeddings/Ankh_Large_Aligned'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"🚀 BẮT ĐẦU QUÁ TRÌNH ALIGNMENT (ANKH3 LARGE)")
print(f"   - Template (Ref): {REF_DIR}")
print(f"   - Source (Raw):   {SRC_DIR}")
print(f"   - Output:         {OUTPUT_DIR}")

# ==============================================================================
# 2. HÀM XỬ LÝ ID (CLEAN ID)
# ==============================================================================
def clean_id(raw_id):
    """Làm sạch ID để khớp format"""
    # Xử lý nếu là bytes
    if isinstance(raw_id, bytes):
        s = raw_id.decode('utf-8')
    else:
        s = str(raw_id)

    # Xử lý b'...'
    if s.startswith("b'") or s.startswith('b"'):
        s = s[2:-1]

    # Cắt bỏ phần thừa (sp|ID|Name -> ID)
    if '|' in s:
        return s.split('|')[1]

    # Cắt bỏ phần space (ID Taxon -> ID)
    return s.split(' ')[0]

# ==============================================================================
# 3. HÀM ALIGN VÀ SAVE (Xử lý từng tập độc lập)
# ==============================================================================
def align_dataset(ref_id_path, src_id_path, src_emb_path, out_emb_name, out_id_name, dataset_name):
    print(f"\n🔄 ĐANG XỬ LÝ TẬP: {dataset_name.upper()}")
    print("-" * 50)

    # --- BƯỚC 1: Load và Index Source IDs ---
    print(f"1. Loading & Indexing Source IDs from: {os.path.basename(src_id_path)}")
    src_raw_ids = np.load(src_id_path, allow_pickle=True)

    # Tạo Map: {Cleaned_ID : Index}
    src_map = {}
    for idx, raw_id in enumerate(tqdm(src_raw_ids, desc=f"Indexing {dataset_name}")):
        clean = clean_id(raw_id)
        src_map[clean] = idx

    print(f"   -> Source contains {len(src_map):,} unique IDs.")

    # Giải phóng raw ids để tiết kiệm RAM
    del src_raw_ids
    gc.collect()

    # --- BƯỚC 2: Load Reference IDs (Thứ tự chuẩn) ---
    print(f"2. Loading Reference IDs from: {os.path.basename(ref_id_path)}")
    ref_ids = np.load(ref_id_path, allow_pickle=True)
    print(f"   -> Reference requires {len(ref_ids):,} IDs.")

    # --- BƯỚC 3: Map IDs (Tìm vị trí) ---
    print("3. Mapping Reference IDs to Source Indices...")
    valid_indices = []
    valid_ids = [] # Lưu lại ID đã clean theo thứ tự
    missing_count = 0

    for pid in tqdm(ref_ids, desc=f"Mapping {dataset_name}"):
        pid_clean = clean_id(pid)

        if pid_clean in src_map:
            valid_indices.append(src_map[pid_clean])
            valid_ids.append(pid_clean)
        else:
            missing_count += 1

    print(f"   -> Matched: {len(valid_indices):,} | Missing: {missing_count}")

    if len(valid_indices) == 0:
        print("❌ LỖI: Không tìm thấy ID nào trùng khớp! Kiểm tra lại dữ liệu nguồn.")
        return

    # --- BƯỚC 4: Extract Embeddings (Dùng mmap) ---
    print(f"4. Extracting Embeddings from: {os.path.basename(src_emb_path)}")
    # Load mmap read-only
    src_emb = np.load(src_emb_path, mmap_mode='r')

    # Check dimensions
    print(f"   -> Source Shape: {src_emb.shape}")

    # Trích xuất (Copy dữ liệu cần thiết vào RAM)
    extracted_emb = src_emb[valid_indices]

    # --- BƯỚC 5: Save ---
    out_emb_path = f"{OUTPUT_DIR}/{out_emb_name}"
    out_id_path = f"{OUTPUT_DIR}/{out_id_name}"

    print(f"5. Saving to {OUTPUT_DIR}...")
    np.save(out_emb_path, extracted_emb)
    np.save(out_id_path, np.array(valid_ids))

    print(f"✅ DONE {dataset_name}! Saved Shape: {extracted_emb.shape}")

    # Dọn dẹp RAM triệt để
    del src_emb, extracted_emb, src_map, valid_indices, valid_ids, ref_ids
    gc.collect()

# ==============================================================================
# 4. CHẠY THỰC TẾ
# ==============================================================================

# --- A. XỬ LÝ TẬP TRAIN ---
align_dataset(
    ref_id_path  = f"{REF_DIR}/ankh_train_FINAL_ids.npy",           # ID Chuẩn
    src_id_path  = f"{SRC_DIR}/train_sequences_ids.npy",            # ID Nguồn Ankh3
    src_emb_path = f"{SRC_DIR}/train_sequences_emb.npy",# Emb Nguồn Ankh3
    out_emb_name = "ankh_large_train_FINAL_embeddings.npy",        # Tên file mới
    out_id_name  = "ankh_large_train_FINAL_ids.npy",
    dataset_name = "TRAIN"
)

print("\n" + "="*50 + "\n")

# --- B. XỬ LÝ TẬP TEST ---
align_dataset(
    ref_id_path  = f"{REF_DIR}/ankh_test_FINAL_ids.npy",            # ID Chuẩn
    src_id_path  = f"{SRC_DIR}/testsuperset_ids.npy",               # ID Nguồn Ankh3
    src_emb_path = f"{SRC_DIR}/testsuperset_emb.npy",   # Emb Nguồn Ankh3
    out_emb_name = "ankh_large_test_FINAL_embeddings.npy",         # Tên file mới
    out_id_name  = "ankh_large_test_FINAL_ids.npy",
    dataset_name = "TEST"
)

print("\n" + "="*50)
print("🎉 HOÀN TẤT TOÀN BỘ!")
print(f"📂 Kiểm tra folder: {OUTPUT_DIR}")

🚀 BẮT ĐẦU QUÁ TRÌNH ALIGNMENT (ANKH3 LARGE)
   - Template (Ref): /content/drive/MyDrive/CAFA6_Embeddings/Final_Merged
   - Source (Raw):   /content/drive/MyDrive/cafa/features/ankh_large/
   - Output:         /content/drive/MyDrive/CAFA6_Embeddings/Ankh_Large_Aligned

🔄 ĐANG XỬ LÝ TẬP: TRAIN
--------------------------------------------------
1. Loading & Indexing Source IDs from: train_sequences_ids.npy


Indexing TRAIN:   0%|          | 0/82404 [00:00<?, ?it/s]

   -> Source contains 82,404 unique IDs.
2. Loading Reference IDs from: ankh_train_FINAL_ids.npy
   -> Reference requires 82,404 IDs.
3. Mapping Reference IDs to Source Indices...


Mapping TRAIN:   0%|          | 0/82404 [00:00<?, ?it/s]

   -> Matched: 82,404 | Missing: 0
4. Extracting Embeddings from: train_sequences_emb.npy
   -> Source Shape: (82404, 1536)
5. Saving to /content/drive/MyDrive/CAFA6_Embeddings/Ankh_Large_Aligned...
✅ DONE TRAIN! Saved Shape: (82404, 1536)



🔄 ĐANG XỬ LÝ TẬP: TEST
--------------------------------------------------
1. Loading & Indexing Source IDs from: testsuperset_ids.npy


Indexing TEST:   0%|          | 0/224309 [00:00<?, ?it/s]

   -> Source contains 224,309 unique IDs.
2. Loading Reference IDs from: ankh_test_FINAL_ids.npy
   -> Reference requires 224,309 IDs.
3. Mapping Reference IDs to Source Indices...


Mapping TEST:   0%|          | 0/224309 [00:00<?, ?it/s]

   -> Matched: 224,309 | Missing: 0
4. Extracting Embeddings from: testsuperset_emb.npy
   -> Source Shape: (224309, 1536)
5. Saving to /content/drive/MyDrive/CAFA6_Embeddings/Ankh_Large_Aligned...
✅ DONE TEST! Saved Shape: (224309, 1536)

🎉 HOÀN TẤT TOÀN BỘ!
📂 Kiểm tra folder: /content/drive/MyDrive/CAFA6_Embeddings/Ankh_Large_Aligned


In [ ]:
import numpy as np
import os

# ==============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ==============================================================================
# 1. Folder Chuẩn (Reference - Ankh Base)
REF_DIR = '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged'

# 2. Folder Cần Check (Target - Ankh3 Large vừa align)
TARGET_DIR = '/content/drive/MyDrive/CAFA6_Embeddings/Ankh_Large_Aligned'

print("🚀 BẮT ĐẦU KIỂM TRA ĐỒNG BỘ DỮ LIỆU")
print("="*60)

# ==============================================================================
# HÀM KIỂM TRA
# ==============================================================================
def check_sync(dataset_name, ref_filename, target_filename):
    print(f"\n🔍 ĐANG KIỂM TRA TẬP: {dataset_name}")
    print("-" * 40)

    ref_path = f"{REF_DIR}/{ref_filename}"
    target_path = f"{TARGET_DIR}/{target_filename}"

    try:
        # Load dữ liệu
        ids_ref = np.load(ref_path, allow_pickle=True)
        ids_target = np.load(target_path, allow_pickle=True)

        # 1. Kiểm tra số lượng
        len_ref = len(ids_ref)
        len_target = len(ids_target)
        print(f"   📊 Số lượng dòng:")
        print(f"      - Chuẩn:  {len_ref:,}")
        print(f"      - Target: {len_target:,}")

        if len_ref != len_target:
            print("   ❌ LỖI: Sai lệch số lượng!")
            return

        # 2. Kiểm tra nội dung (So khớp từng phần tử)
        is_equal = np.array_equal(ids_ref, ids_target)

        if is_equal:
            print("   ✅ KẾT QUẢ: ĐỒNG BỘ HOÀN HẢO (100% MATCH)!")
        else:
            print("   ⚠️ CẢNH BÁO: Số lượng khớp nhưng THỨ TỰ hoặc NỘI DUNG sai!")
            # Tìm vị trí sai đầu tiên
            diff_idx = np.where(ids_ref != ids_target)[0]
            if len(diff_idx) > 0:
                first = diff_idx[0]
                print(f"      -> Sai lệch tại index {first}:")
                print(f"      -> Ref:    {ids_ref[first]}")
                print(f"      -> Target: {ids_target[first]}")

        # 3. In mẫu kiểm chứng mắt thường
        print(f"\n   👀 Mẫu 3 dòng đầu:")
        print(f"      Ref:    {ids_ref[:3]}")
        print(f"      Target: {ids_target[:3]}")

    except FileNotFoundError:
        print(f"   ❌ Lỗi: Không tìm thấy file tại {target_path}")
    except Exception as e:
        print(f"   ❌ Lỗi lạ: {e}")

# ==============================================================================
# CHẠY KIỂM TRA
# ==============================================================================

# 1. Check Train IDs
check_sync(
    dataset_name="TRAIN SET",
    ref_filename="ankh_train_FINAL_ids.npy",
    target_filename="ankh_large_train_FINAL_ids.npy"
)

# 2. Check Test IDs
check_sync(
    dataset_name="TEST SET",
    ref_filename="ankh_test_FINAL_ids.npy",
    target_filename="ankh_large_test_FINAL_ids.npy"
)

print("\n" + "="*60)
print("🏁 HOÀN TẤT.")

🚀 BẮT ĐẦU KIỂM TRA ĐỒNG BỘ DỮ LIỆU

🔍 ĐANG KIỂM TRA TẬP: TRAIN SET
----------------------------------------
   📊 Số lượng dòng:
      - Chuẩn:  82,404
      - Target: 82,404
   ✅ KẾT QUẢ: ĐỒNG BỘ HOÀN HẢO (100% MATCH)!

   👀 Mẫu 3 dòng đầu:
      Ref:    ['A0JNW5' 'A6BM72' 'A6H8Y1']
      Target: ['A0JNW5' 'A6BM72' 'A6H8Y1']

🔍 ĐANG KIỂM TRA TẬP: TEST SET
----------------------------------------
   📊 Số lượng dòng:
      - Chuẩn:  224,309
      - Target: 224,309
   ✅ KẾT QUẢ: ĐỒNG BỘ HOÀN HẢO (100% MATCH)!

   👀 Mẫu 3 dòng đầu:
      Ref:    ['A0JNW5' 'A6BM72' 'A6H8Y1']
      Target: ['A0JNW5' 'A6BM72' 'A6H8Y1']

🏁 HOÀN TẤT.


In [ ]:
import numpy as np
import os

# ==============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ==============================================================================
FILE_PATH = '/content/drive/MyDrive/cafa/features/ankh_large/train_sequences_emb.npy'

def inspect_embedding(path):
    print(f"📂 ĐANG KIỂM TRA FILE: {os.path.basename(path)}")
    print("-" * 60)

    if not os.path.exists(path):
        print(f"❌ Lỗi: Không tìm thấy file tại {path}")
        return

    try:
        # 1. Load chế độ Read-Only (Không tốn RAM)
        data = np.load(path, mmap_mode='r')

        # 2. Thông tin cơ bản
        print(f"✅ Load thành công!")
        print(f"   Shape (Kích thước): {data.shape}")
        print(f"     -> Số lượng Protein: {data.shape[0]:,}")
        print(f"     -> Số chiều Embedding (Dim): {data.shape[1]}")
        print(f"   Dtype (Kiểu dữ liệu): {data.dtype}")

        # 3. Lấy mẫu dữ liệu (Sample)
        # Chỉ lấy protein đầu tiên để xem
        sample_idx = 0
        sample_vec = data[sample_idx]

        print(f"\n🔍 MẪU DỮ LIỆU (Protein tại index {sample_idx}):")
        print(f"   10 giá trị đầu tiên: {sample_vec[:10]}")
        print(f"   Min: {np.min(sample_vec):.4f}")
        print(f"   Max: {np.max(sample_vec):.4f}")
        print(f"   Mean: {np.mean(sample_vec):.4f}")

        # 4. Kiểm tra nhanh lỗi NaN/Inf (trên một tập con nhỏ)
        # Lấy 1000 dòng đầu để check cho nhanh
        subset = data[:1000]
        if np.isnan(subset).any():
            print("\n⚠️ CẢNH BÁO: Dữ liệu chứa giá trị NaN (Not a Number)!")
        elif np.isinf(subset).any():
            print("\n⚠️ CẢNH BÁO: Dữ liệu chứa giá trị Vô cực (Inf)!")
        else:
            print("\n✅ Kiểm tra nhanh 1000 dòng đầu: Không có lỗi NaN/Inf.")

    except Exception as e:
        print(f"\n❌ Có lỗi xảy ra khi đọc file: {e}")

# ==============================================================================
# CHẠY
# ==============================================================================
inspect_embedding(FILE_PATH)

📂 ĐANG KIỂM TRA FILE: train_sequences_emb.npy
------------------------------------------------------------
✅ Load thành công!
   Shape (Kích thước): (82404, 1536)
     -> Số lượng Protein: 82,404
     -> Số chiều Embedding (Dim): 1536
   Dtype (Kiểu dữ liệu): float16

🔍 MẪU DỮ LIỆU (Protein tại index 0):
   10 giá trị đầu tiên: [nan nan nan nan nan nan nan nan nan nan]
   Min: nan
   Max: nan
   Mean: nan

⚠️ CẢNH BÁO: Dữ liệu chứa giá trị NaN (Not a Number)!


In [ ]:
import numpy as np
import os

FILE_PATH = '/content/drive/MyDrive/cafa/features/ankh3_large/train_sequences_emb.npy'

data = np.load(FILE_PATH, mmap_mode='r')

# 2. Thông tin cơ bản
print(f"✅ Load thành công!")
print(f"   Shape (Kích thước): {data.shape}")
print(f"     -> Số lượng Protein: {data.shape[0]:,}")
print(f"     -> Số chiều Embedding (Dim): {data.shape[1]}")
print(f"   Dtype (Kiểu dữ liệu): {data.dtype}")

sample_idx = 0
sample_vec = data[sample_idx]

print(f"\n🔍 MẪU DỮ LIỆU (Protein tại index {sample_idx}):")
print(f"   10 giá trị đầu tiên: {sample_vec[:10]}")

✅ Load thành công!
   Shape (Kích thước): (82404, 1536)
     -> Số lượng Protein: 82,404
     -> Số chiều Embedding (Dim): 1536
   Dtype (Kiểu dữ liệu): float16

🔍 MẪU DỮ LIỆU (Protein tại index 0):
   10 giá trị đầu tiên: [ 1.4786e-02  9.1553e-03 -9.7122e-03 -1.6403e-02 -8.0338e-03  3.2272e-03
  2.5497e-02 -4.5896e-06 -1.7023e-03 -4.5654e-02]
